# Asyncio:
This is a technique used in I/O intensive coding. API calls, network requests etc where concurrency is required.  

### 1_basic.py

A simple idea is to 
1. Write all the funtions with async: async def foo():
2. Call the function with await foo()
3. Run the async with asyncio.run(foo())


### 2_aiohttp.py
It is a basic idea for http call.
1. Create a (one) session with : async with aiohttp.ClientSession() as session:
2. Do the get call with async with session.get(url) as response:
Also focus on the fact that everything in async:
```{python, tidy=FALSE, eval=FALSE, highlight=FALSE }

    async def basic_aiohttp_session(): 
            async with aiohttp.ClientSession() as session: 
                async with session.get(url) as response: 
                    result = await response.json() 
                    ```